<a href="https://colab.research.google.com/github/kato1329/CATech/blob/main/AlexNetsample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
from torch import nn

class CNNblock(nn.Module):
  def __init__(
      self,
      input_channel:int
  ) -> None:
    super().__init__()
    self.conv1 = nn.Conv2d(3,96,kernel_size = (11,11))
    self.pool1 = nn.MaxPool2d(kernel_size = (3,3),stride = 2)
    self.conv2 = nn.Conv2d(96,256,kernel_size = (5,5))
    self.pool2 = nn.MaxPool2d(kernel_size = (3,3),stride = 2)
    self.conv3 = nn.Conv2d(256,384,kernel_size = (3,3))
    self.conv4 = nn.Conv2d(384,384,kernel_size = (3,3))
    self.conv5 = nn.Conv2d(384,256,kernel_size = (3,3))
    self.pool5  = nn.MaxPool2d(kernel_size = (3,3),stride = 2)
    self.flatten = nn.Flatten()
  def forward(
      self,
      input:torch.Tensor
  ) -> torch.Tensor:
    x = self.pool1(self.conv1(input))
    x = self.pool2(self.conv2(x))
    x = self.conv3(x)
    x = self.conv4(x)
    x = self.pool5(self.conv5(x))
    output = self.flatten(x)
    return output
class FFNblock(nn.Module):
  def __init__(
      self,
      input_features:int,
      out_features:int,
      )->None:
        super().__init__()
        self.linear1 = nn.Linear(input_features,4096)
        self.linear2 = nn.Linear(4096,4096)
        self.linear3 = nn.Linear(4096,out_features)
        self.relu = nn.ReLU(inplace = False)
        self.softmax = nn.Softmax(dim = None)
  def forward(
      self,
      input:torch.Tensor
  ) -> torch.Tensor:
    x = self.relu(self.linear1(input))
    x = self.relu(self.linear2(x))
    x = self.relu(self.linear3(x))
    output = self.softmax(x)
    return output

In [9]:
print(CNNblock(3))

CNNblock(
  (conv1): Conv2d(3, 96, kernel_size=(11, 11), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=(3, 3), stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=(3, 3), stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1))
  (conv5): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1))
  (pool5): MaxPool2d(kernel_size=(3, 3), stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
)


In [3]:
class AlexNetModule(nn.Module):
  def __init__(
      self,
      cnn_in:int,
      ffn_in:int,
      ffn_out:int
  ) -> None:
    super().__init__()
    self.cnn = CNNblock(cnn_in)
    self.ffn = FFNblock(ffn_in,ffn_out)
  def forward(
      self,
      input:torch.Tensor
  ) -> torch.Tensor:
    x = self.cnn(input)
    output = self.ffn(x)
    return output

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [20]:
model = AlexNetModule(
    cnn_in = 3,
    ffn_in = 112896,
    ffn_out = 10
).to(device)

In [21]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.05)

In [22]:
###モデルの次元が正しいかどうか確かめるターン
###モデルに224×224のRGB画像を食わせて動かしてみる
import numpy as np
import matplotlib.pyplot as plt

test_image = np.random.rand(1,3,224,224)*100
test_label = np.zeros((1,10))
test_label[0][0] = 1

In [23]:
x = torch.from_numpy(test_image.astype(np.float32)).clone()
y = torch.from_numpy(test_label.astype(np.float32)).clone()

In [28]:
input = x.to(device)
target = y.to(device)
optimizer.zero_grad()
output = model(input)
loss = criterion(output,target)
print("outputの形状は{}".format(output.shape))
print("outputの値は{}".format(output[0]))
print("lossは{}".format(loss))

outputの形状はtorch.Size([1, 10])
outputの値はtensor([0.0861, 0.1181, 0.0861, 0.0861, 0.0861, 0.0861, 0.0861, 0.0861, 0.0861,
        0.1935], device='cuda:0', grad_fn=<SelectBackward0>)
lossは2.317073345184326


<ipython-input-2-900158c14082>:49: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(x)
